In [2]:
import optuna
import lightgbm as lgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

def objective(trial):
    # Define hyperparameter search space
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'verbosity': -1  # Control verbosity
    }

    # Create dataset for LightGBM
    train_data = lgb.Dataset(X_train, label=y_train)

    # Train model without verbose_eval
    gbm = lgb.train(params, train_data, valid_sets=[train_data])

    # Predict
    preds = gbm.predict(X_test)
    preds = (preds > 0.5).astype(int)

    # Evaluate accuracy
    accuracy = accuracy_score(y_test, preds)
    return accuracy

# Create study and optimize
study = optuna.create_study(direction='maximize')  # Maximizing accuracy
study.optimize(objective, n_trials=100)

# Get best hyperparameters
print("Best hyperparameters:", study.best_params)


[I 2024-09-14 08:09:34,799] A new study created in memory with name: no-name-0f1270ef-1841-43f0-87ff-9e115d1a1680
/var/folders/2t/qcxd330n4jn2wp4lyhdx9fwc0000gn/T/ipykernel_37572/1129385781.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
/var/folders/2t/qcxd330n4jn2wp4lyhdx9fwc0000gn/T/ipykernel_37572/1129385781.py:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
/var/folders/2t/qcxd330n4jn2wp4lyhdx9fwc0000gn/T/ipykernel_37572/1129385781.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be r

Best hyperparameters: {'learning_rate': 0.08768342129388511, 'num_leaves': 287, 'max_depth': 11, 'min_data_in_leaf': 28, 'lambda_l1': 2.919207275803013e-08, 'lambda_l2': 0.017540936564006515, 'feature_fraction': 0.6333005249427911}
